In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

d:\Code\Learning\AI\rag-intro\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 2 PDF files to process

Processing: ML intro new one.pdf
  ✓ Loaded 237 pages

Processing: ML intro.pdf
  ✓ Loaded 20 pages

Total documents loaded: 257


In [2]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs
chunks=split_documents(all_pdf_documents)
chunks

Split 257 documents into 703 chunks

Example chunk:
Content: arXiv:1709.02840v3  [cs.LG]  17 May 2018
A Brief Introduction to Machine
Learning for Engineers
(2018), “A Brief Introduction to Machine Learning for Engin eers”, :
V ol. XX, No. XX, pp 1– 231. DOI: X...
Metadata: {'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-05-20T20:15:18-04:00', 'moddate': '2018-05-20T20:15:18-04:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '..\\data\\pdf\\ML intro new one.pdf', 'total_pages': 237, 'page': 0, 'page_label': 'i', 'source_file': 'ML intro new one.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-05-20T20:15:18-04:00', 'moddate': '2018-05-20T20:15:18-04:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '..\\data\\pdf\\ML intro new one.pdf', 'total_pages': 237, 'page': 0, 'page_label': 'i', 'source_file': 'ML intro new one.pdf', 'file_type': 'pdf'}, page_content='arXiv:1709.02840v3  [cs.LG]  17 May 2018\nA Brief Introduction to Machine\nLearning for Engineers\n(2018), “A Brief Introduction to Machine Learning for Engin eers”, :\nV ol. XX, No. XX, pp 1– 231. DOI: XXX.\nOsvaldo Simeone\nDepartment of Informatics\nKing’s College London\nosvaldo.simeone@kcl.ac.uk'),
 Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-05-20T20:15:18-04:00', 'moddate': '2018-05-20T20:15:18-04:00', 'title': '', 'subject': '', 'author': '', 'keywords'

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [5]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [6]:
chunks

[Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-05-20T20:15:18-04:00', 'moddate': '2018-05-20T20:15:18-04:00', 'title': '', 'subject': '', 'author': '', 'keywords': '', 'source': '..\\data\\pdf\\ML intro new one.pdf', 'total_pages': 237, 'page': 0, 'page_label': 'i', 'source_file': 'ML intro new one.pdf', 'file_type': 'pdf'}, page_content='arXiv:1709.02840v3  [cs.LG]  17 May 2018\nA Brief Introduction to Machine\nLearning for Engineers\n(2018), “A Brief Introduction to Machine Learning for Engin eers”, :\nV ol. XX, No. XX, pp 1– 231. DOI: XXX.\nOsvaldo Simeone\nDepartment of Informatics\nKing’s College London\nosvaldo.simeone@kcl.ac.uk'),
 Document(metadata={'producer': 'dvips + GPL Ghostscript GIT PRERELEASE 9.22', 'creator': 'LaTeX with hyperref package', 'creationdate': '2018-05-20T20:15:18-04:00', 'moddate': '2018-05-20T20:15:18-04:00', 'title': '', 'subject': '', 'author': '', 'keywords'

In [8]:
### convert the text
texts = [doc.page_content for doc in chunks]


In [ ]:
## Generate embedding 
embeddings = embedding_manager.generate_embeddings(texts)



Generating embeddings for 703 texts...


Batches: 100%|██████████| 22/22 [00:11<00:00,  1.87it/s]

Generated embeddings with shape: (703, 384)


In [10]:
embeddings

array([[-0.06859832, -0.0196328 ,  0.13569172, ...,  0.04772124,
        -0.10425296,  0.04991322],
       [ 0.01080389,  0.04381944, -0.02950113, ...,  0.09828102,
         0.0225188 , -0.02271296],
       [ 0.03615247, -0.05212416, -0.0576537 , ...,  0.02717868,
        -0.04543386,  0.04596977],
       ...,
       [-0.05402548, -0.04097266,  0.04875919, ...,  0.09031661,
        -0.14548384,  0.05893738],
       [-0.06312645, -0.04029313,  0.00369945, ..., -0.02250622,
        -0.11598643, -0.00098798],
       [-0.08754771, -0.01736764, -0.07827104, ..., -0.04052905,
        -0.03786953,  0.07347643]], shape=(703, 384), dtype=float32)

In [11]:
vectorstore.add_documents(chunks,embeddings)


Adding 703 documents to vector store...
Successfully added 703 documents to vector store
Total documents in collection: 703
